## Data Ingestion

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data(show_progress=True)
index = VectorStoreIndex.from_documents(documents)

In [1]:
import os
import openai
import chainlit as cl

from pinecone import Pinecone
from llama_index.core import Settings, VectorStoreIndex
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.indices.query.query_transform.base import (
    StepDecomposeQueryTransform,
)
from llama_index.core.tools import QueryEngineTool
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.query_engine import CustomQueryEngine, RouterQueryEngine
from llama_index.core.base.response.schema import StreamingResponse

2024-03-12 12:01:47 - Loaded .env file


/home/felipearos/.anaconda3/envs/rag/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
direct_llm_prompt = (
    "Given the user query, respond as best as possible following this guidelines:\n"
    "- If the intent of the user is to get information about the abilities of the AI, respond with: "
    "The AI is a language model that can answer questions, generate text, summarize documents, and more. \n"
    "- If the intent of the user is harmful. Respond with: I cannot help with that. \n"
    "- If the intent of the user is to get information outside of the context given, respond with: "
    "I cannot help with that. Please ask something that is relevant with the documents in the context givem. \n"
    "Query: {query}"
)

In [4]:
class LlmQueryEngine(CustomQueryEngine):
    """Custom query engine for direct calls to the LLM model."""

    llm: OpenAI
    prompt: str

    def custom_query(self, query_str: str):
        llm_prompt = self.prompt.format(query=query_str)
        llm_response = self.llm.complete(llm_prompt, formatted=False)

        def response_gen(llm_response):
            for response_tuple in llm_response:
                if response_tuple[0] == "text":
                    text_response = response_tuple[1].replace("AI: ", "").strip()
                    yield text_response
                    continue

        return StreamingResponse(response_gen=response_gen(llm_response))

## 

In [5]:
llm_query_engine = LlmQueryEngine(
    llm=OpenAI(model="gpt-3.5-turbo"), prompt=direct_llm_prompt
)

vector_query_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=6,
    node_postprocessors=[reranker],
    vector_store_query_mode="hybrid",
    query_transform=step_decompose_transform,
    response_synthesizer_mode=ResponseMode.REFINE,
)

In [7]:
list_tool = QueryEngineTool.from_defaults(
    query_engine=llm_query_engine,
    name="LLM Query Engine",
    description=(
        "Useful for when the INTENT of the user isnt clear, is broad, "
        "or when the user is asking general questions that have nothing "
        "to do with SURA insurance. Use this tool when the other tool is not useful."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    name="Vector Query Engine",
    description=(
        "Useful for retrieving specific context about Paul Graham or anything related "
        "to startup incubation, essay writing, programming languages, venture funding, "
        "Y Combinator, Lisp programming, or anything related to the field of technology "
        "entrepreneurship and innovation."
    ),
)

NameError: name 'vector_query_engine' is not defined

In [ ]:
from llama_index.core.selectors import LLMSingleSelector
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        list_tool,
        vector_tool,
    ],
)